#### Feature store creation

In [168]:
import json
import uuid
import boto3
import sagemaker
import pandas as pd

from sagemaker.feature_store.feature_group import (
    FeatureGroup,
    FeatureDefinition,
    FeatureTypeEnum,
)
from sagemaker.feature_store.inputs import TableFormatEnum
from datetime import datetime, timezone, date
import time

In [169]:
s3 = boto3.client("s3")
session = sagemaker.Session()
feature_store_client = session.boto_session.client("sagemaker-featurestore-runtime")
sagemaker_client = session.boto_session.client("sagemaker")
bucket_name = "team1-index-predictor-bucket"
feature_group_name = "index-predictor-feature-group-v7"
feature_store_bucket_prefix = "feature-store-v7"
sm_role = sagemaker.get_execution_role()

In [170]:
def generate_event_timestamp():
    # naive datetime representing local time
    naive_dt = datetime.now()
    # take timezone into account
    aware_dt = naive_dt.astimezone()
    # time in UTC
    utc_dt = aware_dt.astimezone(timezone.utc)
    # transform to ISO-8601 format
    event_time = utc_dt.isoformat(timespec="milliseconds")
    event_time = event_time.replace("+00:00", "Z")
    return event_time

In [171]:
df = pd.read_csv("../data/processed/data.csv")


def convert_col_name(c):
    return c.lower().replace(".", "_").replace("-", "_").rstrip("_")


df = df.rename(columns=convert_col_name)

In [172]:
feature_group = FeatureGroup(
    name=feature_group_name,
    sagemaker_session=session,
)

feature_group.load_feature_definitions(data_frame=df)

print()

In [173]:
feature_group.create(
    s3_uri=f"s3://{bucket_name}/{feature_store_bucket_prefix}",
    record_identifier_name="datetime",
    event_time_feature_name="datetime",
    role_arn=sm_role,
    enable_online_store=False,
    table_format=TableFormatEnum.ICEBERG,
)

{'FeatureGroupArn': 'arn:aws:sagemaker:eu-central-1:567821811420:feature-group/index-predictor-feature-group-v7',
 'ResponseMetadata': {'RequestId': 'a3dfe05c-1eab-4682-92d6-4a656e2bdbcd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a3dfe05c-1eab-4682-92d6-4a656e2bdbcd',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '112',
   'date': 'Fri, 21 Jun 2024 07:15:55 GMT'},
  'RetryAttempts': 0}}

In [174]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    print(f"Initial status: {status}")
    while status == "Creating":
        print(f"Waiting for feature group: {feature_group.name} to be created ...")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}: {status}"
        )
    print(f"FeatureGroup {feature_group.name} was successfully created.")

In [175]:
wait_for_feature_group_creation_complete(feature_group)

feature_group.describe()

Initial status: Creating
Waiting for feature group: index-predictor-feature-group-v7 to be created ...
Waiting for feature group: index-predictor-feature-group-v7 to be created ...
Waiting for feature group: index-predictor-feature-group-v7 to be created ...
Waiting for feature group: index-predictor-feature-group-v7 to be created ...
Waiting for feature group: index-predictor-feature-group-v7 to be created ...
Waiting for feature group: index-predictor-feature-group-v7 to be created ...
FeatureGroup index-predictor-feature-group-v7 was successfully created.


{'FeatureGroupArn': 'arn:aws:sagemaker:eu-central-1:567821811420:feature-group/index-predictor-feature-group-v7',
 'FeatureGroupName': 'index-predictor-feature-group-v7',
 'RecordIdentifierFeatureName': 'datetime',
 'EventTimeFeatureName': 'datetime',
 'FeatureDefinitions': [{'FeatureName': 'datetime', 'FeatureType': 'String'},
  {'FeatureName': 'open', 'FeatureType': 'Fractional'},
  {'FeatureName': 'high', 'FeatureType': 'Fractional'},
  {'FeatureName': 'low', 'FeatureType': 'Fractional'},
  {'FeatureName': 'close', 'FeatureType': 'Fractional'},
  {'FeatureName': 'volume', 'FeatureType': 'Integral'},
  {'FeatureName': 'hour', 'FeatureType': 'Integral'},
  {'FeatureName': 'minute', 'FeatureType': 'Integral'},
  {'FeatureName': 'dayofweek_0', 'FeatureType': 'Integral'},
  {'FeatureName': 'dayofweek_1', 'FeatureType': 'Integral'},
  {'FeatureName': 'dayofweek_2', 'FeatureType': 'Integral'},
  {'FeatureName': 'dayofweek_3', 'FeatureType': 'Integral'},
  {'FeatureName': 'dayofweek_4', 'Fe